<a href="https://colab.research.google.com/github/LABYRINTH3/NALM/blob/main/%EA%B8%B0%ED%95%99%EC%8B%AC_LLaDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U "datasets>=2.19"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun May 25 07:03:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
import os
import glob
import torch
import math
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
from datasets import load_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 0. Settings


In [ ]:
device         = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset_root   = "/content/drive/My Drive/Datasets"
checkpoint_dir = "/content/drive/My Drive/Datasets/checkpoint"
test_dir       = "/content/drive/My Drive/Datasets/test"

os.makedirs(checkpoint_dir, exist_ok=True)

mask_probs       = [round(p,1) for p in torch.arange(0.6, 0.2, -0.1).tolist()]
print(mask_probs)
epochs_per_stage = 2
batch_size       = 16
lr               = 1e-4


[0.6, 0.5, 0.4, 0.3]


In [ ]:
# file_path = "/content/drive/My Drive/Datasets/test_file.txt"
# with open(file_path, "w", encoding="utf-8") as f:
#     f.write("테스트: 파일 저장 성공!\n")
#     f.write("두 번째 줄도 잘 써져요.")

# print(f"✅ 저장 완료: {file_path}")

# 1. Tokenizer

In [ ]:
tokenizer     = AutoTokenizer.from_pretrained("bert-base-uncased")
mask_token_id = tokenizer.mask_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# 2. 모델 정의 (생략, 이전과 동일)

In [ ]:
class MaskedDiffusionTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=512, n_layers=6, n_heads=8, max_length=512):
        super().__init__()
        self.token_emb   = nn.Embedding(vocab_size, d_model)
        self.pos_emb     = nn.Embedding(max_length, d_model)
        enc_layer       = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_heads, batch_first = True)
        self.encoder     = nn.TransformerEncoder(enc_layer, num_layers=n_layers)
        self.output_proj = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids, mask_positions=None):
        B, L = input_ids.shape
        pos = torch.arange(L, device=input_ids.device) \
                 .unsqueeze(0).expand(B, -1)
        x = self.token_emb(input_ids) + self.pos_emb(pos)
        x = self.encoder(x)
        return self.output_proj(x)

criterion = nn.CrossEntropyLoss(reduction='none')
def diffusion_loss(logits, target_ids, mask_positions):
    B, L, V = logits.shape
    loss_all = criterion(
        logits.view(B*L, V),
        target_ids.view(B*L)
    ).view(B, L)
    return (loss_all * mask_positions.float()).sum() \
           / mask_positions.sum().clamp_min(1.0)

# 3. 샤드된 .arrow 파일을 한 번에 불러오는 함수

In [ ]:
def load_sharded_dataset(mask_prob):
    """
    mask_prob : directory "tinystories_masked_p20" 안에 data-00001-of-00008.arrow ... data-00005-of-00007.arrow
    """
    stage_dir = os.path.join(dataset_root, f"tinystories_masked_p{int(mask_prob*100)}")
    # glob으로 8개 샤드 파일 경로 수집
    files = sorted(glob.glob(os.path.join(stage_dir, "*.arrow")))
    if not files:
        raise FileNotFoundError(f"No .arrow files in {stage_dir}")
    # HuggingFace datasets로 로드
    ds = load_dataset("arrow", data_files={"train": files}, split="train")
    # PyTorch 텐서 형식으로 변환
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return ds

# 4. 한 스테이지 학습 함수


In [ ]:
def train_stage(model, dataloader, optimizer, scheduler=None):
    model.train()
    total_loss = 0.0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        labels    = batch["labels"].to(device)
        mask_pos  = labels.ne(-100)   # -100이 아닌 곳이 마스크된 토큰

        logits = model(input_ids)
        loss   = diffusion_loss(logits, labels, mask_pos)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if scheduler: scheduler.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

# 5. Curriculum Training

In [ ]:
def curriculum_train():
    # 모델·옵티마이저 한 번만 생성
    model     = MaskedDiffusionTransformer(tokenizer.vocab_size).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    prev_ckpt = None
    for p in mask_probs:
        print(f"Stage mask_prob = {p:.1f}")
        # 1) 샤드된 arrow 파일 모두 로드
        ds = load_sharded_dataset(p)
        loader = DataLoader(
            ds,
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
            pin_memory=True
        )

        # 2) 이전 스테이지 가중치 로드
        if prev_ckpt:
            model.load_state_dict(torch.load(prev_ckpt, map_location=device))
            print(f"loaded weights from {prev_ckpt}")

        # 3) 스케줄러 설정 (옵션)
        total_steps = epochs_per_stage * len(loader)
        scheduler   = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps = total_steps // 10,
            num_training_steps = total_steps
        )

        # 4) 학습
        avg_loss = train_stage(model, loader, optimizer, scheduler)
        print(f"[mask={p:.1f}] loss: {avg_loss:.4f}")

        # 5) 체크포인트 저장
        ckpt_path = os.path.join(checkpoint_dir, f"model_mask{int(p*100)}.pt")
        torch.save(model.state_dict(), ckpt_path)
        print(f"saved checkpoint: {ckpt_path}")

        prev_ckpt = ckpt_path

    print("\n== Curriculum Training Complete ==")

마무리

In [ ]:
if __name__ == "__main__":
    curriculum_train()

Stage mask_prob = 0.6


Generating train split: 0 examples [00:00, ? examples/s]

[mask=0.6] loss: 2.8815
saved checkpoint: /content/drive/My Drive/Datasets/checkpoint/model_mask60.pt
Stage mask_prob = 0.5


Generating train split: 0 examples [00:00, ? examples/s]

loaded weights from /content/drive/My Drive/Datasets/checkpoint/model_mask60.pt
[mask=0.5] loss: 1.7713
saved checkpoint: /content/drive/My Drive/Datasets/checkpoint/model_mask50.pt
Stage mask_prob = 0.4


...

Generating train split: 0 examples [00:00, ? examples/s]

loaded weights from /content/drive/My Drive/Datasets/checkpoint/model_mask40.pt
[mask=0.3] loss: 1.0832
saved checkpoint: /content/drive/My Drive/Datasets/checkpoint/model_mask30.pt
Stage mask_prob = 0.2


Generating train split: 0 examples [00:00, ? examples/s]

loaded weights from /content/drive/My Drive/Datasets/checkpoint/model_mask30.pt
[mask=0.2] loss: 0.8019
saved checkpoint: /content/drive/My Drive/Datasets/checkpoint/model_mask20.pt
Stage mask_prob = 0.1


FileNotFoundError: No .arrow files in /content/drive/My Drive/Datasets/tinystories_masked_p10

In [ ]:
import glob

# 추론 샘플링

In [ ]:
# 6. Inference: 마스크 채워넣기 단계 구현


def mask_inputs(input_ids, t, mask_token_id, prompt_length):
    """
    input_ids: (B, L) 정수 텐서, 마스크/기존 토큰이 섞여 있는 시퀀스
    t:        (B,) 실수 텐서, 각 배치별 현재 단계의 마스크 확률
    mask_token_id: 마스크 토큰 ID
    prompt_length: 프롬프트 길이 L_p (정수)
    반환:
      noised_inputs: (B, L) 마스크가 적용된 입력
      mask_pos:      (B, L) bool 텐서, 이번 단계에서 마스크된 위치
    """
    B, L = input_ids.shape
    # 1) generation 영역 (프롬프트 뒤)만 마스킹 후보
    gen_region = torch.zeros_like(input_ids, dtype=torch.bool)
    gen_region[:, prompt_length:] = True

    # 2) 각 위치마다 [0,1) 균등 난수 생성해서 t보다 작으면 마스크
    rand = torch.rand((B, L), device=input_ids.device)
    step_mask = rand < t.view(B, 1)

    mask_pos = gen_region & step_mask

    noised = input_ids.clone()
    noised[mask_pos] = mask_token_id
    return noised, mask_pos

def sample_from_model(model, tokenizer, prompt_ids,
                      response_length=50, steps=10, device='cuda'):
    """
    model:           학습된 MaskedDiffusionTransformer
    tokenizer:       AutoTokenizer
    prompt_ids:      (B, L_p) 프롬프트 토큰 ID
    response_length: 생성할 토큰 수 R
    steps:           디퓨전 스텝 수 T
    device:          'cuda' or 'cpu'
    반환: (B, R) 채워넣기 완료된 토큰 ID
    """
    model.eval()
    B, Lp = prompt_ids.shape
    R = response_length

    # 1) 생성부 전부 [MASK]로 초기화
    response = torch.full((B, R),
                          tokenizer.mask_token_id,
                          dtype=torch.long,
                          device=device)
    combined = torch.cat([prompt_ids.to(device), response], dim=1)

    # 2) t를 1→0 선형 스케줄로 감소시키며 반복
    t_schedule = torch.linspace(1.0, 0.0, steps, device=device)
    for step in range(steps):
        t = t_schedule[step].expand(B)  # 현재 단계의 마스크 확률

        # 2-1) 현재 t에 맞춰 일부 토큰을 다시 마스크
        noised_inputs, mask_pos = mask_inputs(
            combined, t, tokenizer.mask_token_id, Lp
        )

        # 2-2) 모델에 feed-forward
        logits = model(noised_inputs)

        # 2-3) 마스크된 위치만 예측 결과로 교체
        preds = logits.argmax(-1)
        combined[mask_pos] = preds[mask_pos]

    # 3) 프롬프트 뒤 생성 부분만 리턴
    return combined[:, Lp:]

if __name__ == "__main__":
    # ── 마지막 체크포인트 자동 탐색 및 로드 ──
    ckpts = sorted(glob.glob(os.path.join(checkpoint_dir, "model_mask*.pt")))
    assert ckpts, "체크포인트 파일을 찾을 수 없습니다."

    for i in range (0,5):
        last_ckpt = ckpts[i]

        # 1) 모델 선언 & 로드
        model = MaskedDiffusionTransformer(tokenizer.vocab_size).to(device)
        model.load_state_dict(torch.load(last_ckpt, map_location=device))
        print(f"Loaded weights from {last_ckpt}")

        # 2) 예시 프롬프트 준비
        example_text = "Once upon a time"
        enc = tokenizer.encode(example_text, return_tensors="pt")

        # 3) 샘플링
        out_ids = sample_from_model(
            model, tokenizer,
            prompt_ids=enc,
            response_length=30,
            steps=40,
            device=device
        )

        # 4) 디코딩 및 출력
        generated = tokenizer.decode(out_ids[0], skip_special_tokens=True)
        print("Generated :")
        print(generated)
        print("")


Loaded weights from /content/drive/My Drive/Datasets/checkpoint/model_mask20.pt
Generated :
, there was a very big test. the test was so big that it could not fit through it. the test was big and strong. in the

Loaded weights from /content/drive/My Drive/Datasets/checkpoint/model_mask30.pt
Generated :
there was a little mouse. the mouse was very curious and loved to explore. one day, he heard loud noises in the forest. he was scared

Loaded weights from /content/drive/My Drive/Datasets/checkpoint/model_mask40.pt
Generated :
there was a little boy who loved to play outside. one day, he went to play with his friend bunny. they had so much fun together!

Loaded weights from /content/drive/My Drive/Datasets/checkpoint/model_mask50.pt
Generated :
person was walking in the park. suddenly they saw a big dog. the dog ran away! the person was very scared, so it ran away.

Loaded weights from /content/drive/My Drive/Datasets/checkpoint/model_mask60.pt
Generated :
there was a big rock. it broke.